In [34]:
from data import get_movielens_1m
from surprise import KNNBasic
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy

In [35]:
import random
import numpy as np

np.random.seed(42)
random.seed(42)

In [36]:
movies_df, users_df, ratings_df = get_movielens_1m()
data = Dataset.load_builtin('ml-100k')

In [37]:
a = ratings_df.groupby('UserID').count().reset_index()
a[a['MovieID'] == 20]

,UserID,MovieID,Rating,Timestamp
97,98,20,20,20
159,160,20,20,20
216,217,20,20,20
249,250,20,20,20
310,311,20,20,20
...,...,...,...,...
5532,5533,20,20,20
5724,5725,20,20,20
5827,5828,20,20,20
5903,5904,20,20,20


In [38]:
import pandas as pd
for measure in ["cosine", "pearson", "msd"]:
    for user_based in [True, False]:
        print(measure, user_based)
        sim_options = {'name': measure, 'user_based': user_based }
        algo = KNNBasic(sim_options=sim_options)
        results = cross_validate(algo, data, ["rmse", "mae"], 5)
        print(pd.DataFrame.from_dict(results))

cosine True
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
   test_rmse  test_mae  fit_time  test_time
0   1.011953  0.798421  0.877009   1.369701
1   1.011719  0.800784  0.890913   1.532922
2   1.023941  0.812346  0.845001   1.371013
3   1.018417  0.806602  0.881015   1.363031
4   1.019354  0.803799  0.847665   1.370006
cosine False
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine simila

In [39]:
train_ds, test_ds = train_test_split(data, test_size=0.1)

alg = KNNBasic()

predictions = alg.fit(train_ds).test(test_ds)

accuracy.rmse(predictions)


Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9717


0.9717303709200743

In [40]:
USER_ID = 98
movies_by = ratings_df[ratings_df['UserID'] == USER_ID]
for idx, row in movies_by.iterrows():
    movie = movies_df.loc[movies_df['MovieID'] == row['MovieID']].iloc[0]
    print(f'Movie: {movie["Title"]}, genre: {movie["Genres"]}, rating: {row["Rating"]}')

Movie: Mission: Impossible (1996), genre: Action|Adventure|Mystery, rating: 4
Movie: Bridge on the River Kwai, The (1957), genre: Drama|War, rating: 5
Movie: Silence of the Lambs, The (1991), genre: Drama|Thriller, rating: 3
Movie: My Cousin Vinny (1992), genre: Comedy, rating: 3
Movie: Class (1983), genre: Comedy, rating: 3
Movie: Gone with the Wind (1939), genre: Drama|Romance|War, rating: 5
Movie: Batman & Robin (1997), genre: Action|Adventure|Crime, rating: 3
Movie: Goodbye Girl, The (1977), genre: Comedy|Romance, rating: 3
Movie: Odd Couple, The (1968), genre: Comedy, rating: 4
Movie: Rocky Horror Picture Show, The (1975), genre: Comedy|Horror|Musical|Sci-Fi, rating: 1
Movie: Shawshank Redemption, The (1994), genre: Drama, rating: 5
Movie: Gladiator (2000), genre: Action|Drama, rating: 5
Movie: Pulp Fiction (1994), genre: Crime|Drama, rating: 4
Movie: Airplane! (1980), genre: Comedy, rating: 5
Movie: Escape from New York (1981), genre: Action|Adventure|Sci-Fi|Thriller, rating: 4
M

In [41]:
from random import randint

def get_prediction(user_id: int, movie_id: int):
    user_id = str(user_id)
    movie_id = str(movie_id)

    rating = alg.predict(user_id, movie_id)
    return rating

for i in range(10):
    uid = USER_ID
    mid = randint(1, 1667)

    rating = get_prediction(uid, mid)

    movie = movies_df.loc[movies_df['MovieID'] == mid].iloc[0]
    print(f'Movie: {movie["Title"]}, genre: {movie["Genres"]}, rating: {rating.est}, {rating.details}')

Movie: Hype! (1996), genre: Documentary, rating: 3.0, {'actual_k': 1, 'was_impossible': False}
Movie: Death and the Maiden (1994), genre: Drama|Thriller, rating: 3.180728474668971, {'actual_k': 40, 'was_impossible': False}
Movie: Mighty Aphrodite (1995), genre: Comedy, rating: 3.8938345371196608, {'actual_k': 40, 'was_impossible': False}
Movie: Broken English (1996), genre: Drama, rating: 3.460674157303371, {'actual_k': 2, 'was_impossible': False}
Movie: Chasers (1994), genre: Comedy, rating: 1.978944209802029, {'actual_k': 21, 'was_impossible': False}
Movie: Next Karate Kid, The (1994), genre: Action|Children's, rating: 3.8582063423188435, {'actual_k': 40, 'was_impossible': False}
Movie: Geronimo: An American Legend (1993), genre: Drama|Western, rating: 2.926599348794026, {'actual_k': 40, 'was_impossible': False}
Movie: Nemesis 2: Nebula (1995), genre: Action|Sci-Fi|Thriller, rating: 3.775, {'actual_k': 40, 'was_impossible': False}
Movie: All Over Me (1997), genre: Drama, rating: 1.60